In [1]:
!nvidia-smi

Mon May  9 20:14:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers==4.18.0 
!pip install datasets

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
import datasets
import warnings # to ignore warnings

warnings.filterwarnings('ignore')

In [5]:
language = 'zh'
pretraining_data_path = f"./squad.pretraining.{language}.csv"

In [6]:
from transformers import AutoTokenizer, LineByLineTextDataset

# model_name = "microsoft/Multilingual-MiniLM-L12-H384"
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path=pretraining_data_path, block_size=128)

In [7]:
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer, AutoModelForMaskedLM

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True, mlm_probability=0.15)    

 
training_args = TrainingArguments(
    output_dir=f"{language}-TAPT-MLM-MiniLM",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=16,
    save_total_limit=1,
    push_to_hub=True,
    fp16=True
)

In [8]:
model_name = "subhasisj/MiniLMv2-qa-encoder"
model = AutoModelForMaskedLM.from_pretrained(model_name)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset)

trainer.train()

/content/zh-TAPT-MLM-MiniLM is already a clone of https://huggingface.co/subhasisj/zh-TAPT-MLM-MiniLM. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 85701
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5357
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,5.897900
1000,4.174000
1500,3.868300
2000,3.697400
2500,3.585700
3000,3.488400
3500,3.436600
4000,3.374200
4500,3.363000
5000,3.330000


Saving model checkpoint to zh-TAPT-MLM-MiniLM/checkpoint-500
Configuration saved in zh-TAPT-MLM-MiniLM/checkpoint-500/config.json
Model weights saved in zh-TAPT-MLM-MiniLM/checkpoint-500/pytorch_model.bin
Saving model checkpoint to zh-TAPT-MLM-MiniLM/checkpoint-1000
Configuration saved in zh-TAPT-MLM-MiniLM/checkpoint-1000/config.json
Model weights saved in zh-TAPT-MLM-MiniLM/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [zh-TAPT-MLM-MiniLM/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to zh-TAPT-MLM-MiniLM/checkpoint-1500
Configuration saved in zh-TAPT-MLM-MiniLM/checkpoint-1500/config.json
Model weights saved in zh-TAPT-MLM-MiniLM/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [zh-TAPT-MLM-MiniLM/checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to zh-TAPT-MLM-MiniLM/checkpoint-2000
Configuration saved in zh-TAPT-MLM-MiniLM/checkpoint-2000/config.json
Model weights saved in zh-TAPT-MLM-MiniLM/checkpoint-2000/pytorc

TrainOutput(global_step=5357, training_loss=3.787743060518072, metrics={'train_runtime': 10664.5558, 'train_samples_per_second': 8.036, 'train_steps_per_second': 0.502, 'total_flos': 1427796606475008.0, 'train_loss': 3.787743060518072, 'epoch': 1.0})

In [10]:
trainer.push_to_hub('Training Completed')

Saving model checkpoint to zh-TAPT-MLM-MiniLM
Configuration saved in zh-TAPT-MLM-MiniLM/config.json
Model weights saved in zh-TAPT-MLM-MiniLM/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/450M [00:00<?, ?B/s]

Upload file runs/May09_20-15-33_9901267adec5/events.out.tfevents.1652127343.9901267adec5.1175.0:  65%|######4 …

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/subhasisj/zh-TAPT-MLM-MiniLM
   2cbe70b..4f91ef2  main -> main

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/subhasisj/zh-TAPT-MLM-MiniLM
   4f91ef2..10011a5  main -> main



'https://huggingface.co/subhasisj/zh-TAPT-MLM-MiniLM/commit/4f91ef2d932198a19f7de13feb1c1a22f1bb0322'